# Ansible

`pip3 install ansible`

In [6]:
import docker

docker_client = docker.from_env()

network = docker_client.networks.create(
    name="ansible_example",
    ipam=docker.types.IPAMConfig(
        pool_configs=[
            docker.types.IPAMPool(
                subnet="192.0.2.0/24"
            )
        ]
    )
)

containers = []

for i, ip in enumerate(["192.0.2.50", "192.0.2.51", "192.0.2.52"]):

    container = docker_client.api.create_container(
        image="alpine", 
        networking_config=docker_client.api.create_networking_config({
            network.name: docker_client.api.create_endpoint_config(
            ipv4_address=ip
        )}),
        name=f"ansible_example{i}",
        stdin_open=True,
        detach=True,
        tty=True
    )

    docker_client.api.start(container['Id'])
    containers.append(container)

In [39]:
%%writefile ansible_files/inventory.ini
[myhosts]
192.0.2.50
192.0.2.51
192.0.2.52

Overwriting ansible_files/inventory.ini


In [40]:
!ansible myhosts -m ping -i ansible_files/inventory.ini

192.0.2.51 | UNREACHABLE! => {
    "changed": false,
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 192.0.2.51 port 22: Connection timed out",
    "unreachable": true
}
192.0.2.50 | UNREACHABLE! => {
    "changed": false,
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 192.0.2.50 port 22: Connection timed out",
    "unreachable": true
}
192.0.2.52 | UNREACHABLE! => {
    "changed": false,
    "msg": "Failed to connect to the host via ssh: ssh: connect to host 192.0.2.52 port 22: Connection timed out",
    "unreachable": true
}


In [38]:
for container in containers:
    docker_client.api.stop(container=container["Id"])
    docker_client.api.remove_container(container=container["Id"])
network.remove()